<a href="https://colab.research.google.com/github/ImNotDanish05/2025_KC_TI-1A/blob/main/P11_Convolutional%20Neural%20Network%20(CNN).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [17]:
import tensorflow as tf
import os
print(tf.__version__)

2.18.0


In [9]:
"""
List Dir: https://api.github.com/repos/ImNotDanish05/2025_KC_TI-1A/contents/Data/P11/images
Open File: https://raw.githubusercontent.com/ImNotDanish05/2025_KC_TI-1A/main/Data/P11/images
"""

base_dir = 'https://api.github.com/repos/ImNotDanish05/2025_KC_TI-1A/contents/Data/P11/images'
train_dir = base_dir + '/train'
validation_dir = base_dir + '/val'

In [27]:
import requests
from urllib.parse import urlparse
def listdirx(dirx):
  response = requests.get(dirx)
  if response.status_code == 200:
      files = response.json()
      print(dirx)
      for file in files:
          print(f"- {file['name']}")
  else:
      print(f"Gagal mengambil data. Status: {response.status_code}")
def list_all_png_from_github_api(api_url, collected=[]):
    response = requests.get(api_url)
    if response.status_code != 200:
        print(f"❌ Gagal mengakses {api_url} | Status: {response.status_code}")
        return collected

    items = response.json()
    for item in items:
        if item['type'] == 'file' and item['name'].endswith('.png'):
            collected.append(item['download_url'])
        elif item['type'] == 'dir':
            list_all_png_from_github_api(item['url'], collected)
    return collected
def download_images(image_urls, save_dir):
    os.makedirs(save_dir, exist_ok=True)
    for url in image_urls:
        filename = os.path.basename(urlparse(url).path)
        save_path = os.path.join(save_dir, filename)

        response = requests.get(url)
        if response.status_code == 200:
            with open(save_path, 'wb') as f:
                f.write(response.content)
            print(f"✅ Downloaded: {filename}")
        else:
            print(f"❌ Gagal download: {filename}")

In [24]:
listdirx(train_dir)
listdirx(validation_dir)

https://api.github.com/repos/ImNotDanish05/2025_KC_TI-1A/contents/Data/P11/images/train
- .DS_Store
- clean
- messy
https://api.github.com/repos/ImNotDanish05/2025_KC_TI-1A/contents/Data/P11/images/val
- clean
- messy


TensorFlow

Documentation: https://keras.io/api/data_loading/image/

In [28]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
                    rescale=1./255,
                    rotation_range=20,
                    horizontal_flip=True,
                    shear_range = 0.2,
                    fill_mode = 'nearest')

test_datagen = ImageDataGenerator(
                    rescale=1./255)
train_clean_urls = list_all_png_from_github_api(train_dir + "/clean")
train_messy_urls = list_all_png_from_github_api(train_dir + "/messy")
download_images(train_clean_urls, 'dataset/train/clean')
download_images(train_messy_urls, 'dataset/train/messy')

['https://raw.githubusercontent.com/ImNotDanish05/2025_KC_TI-1A/main/Data/P11/images/train/clean/0.png',
 'https://raw.githubusercontent.com/ImNotDanish05/2025_KC_TI-1A/main/Data/P11/images/train/clean/1.png',
 'https://raw.githubusercontent.com/ImNotDanish05/2025_KC_TI-1A/main/Data/P11/images/train/clean/10.png',
 'https://raw.githubusercontent.com/ImNotDanish05/2025_KC_TI-1A/main/Data/P11/images/train/clean/11.png',
 'https://raw.githubusercontent.com/ImNotDanish05/2025_KC_TI-1A/main/Data/P11/images/train/clean/12.png',
 'https://raw.githubusercontent.com/ImNotDanish05/2025_KC_TI-1A/main/Data/P11/images/train/clean/13.png',
 'https://raw.githubusercontent.com/ImNotDanish05/2025_KC_TI-1A/main/Data/P11/images/train/clean/14.png',
 'https://raw.githubusercontent.com/ImNotDanish05/2025_KC_TI-1A/main/Data/P11/images/train/clean/15.png',
 'https://raw.githubusercontent.com/ImNotDanish05/2025_KC_TI-1A/main/Data/P11/images/train/clean/16.png',
 'https://raw.githubusercontent.com/ImNotDanish0

In [26]:
train_generator = train_datagen.flow_from_directory(
        train_dir,  # direktori data latih
        target_size=(150, 150),  # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4,
        # karena ini merupakan masalah klasifikasi 2 kelas, gunakan class_mode = 'binary'
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir, # direktori data validasi
        target_size=(150, 150), # mengubah resolusi seluruh gambar menjadi 150x150 piksel
        batch_size=4, # karena ini merupakan masalah klasifikasi 2 kelas gunakan class_mode = 'binary'
        class_mode='binary')

TypeError: listdir: path should be string, bytes, os.PathLike, integer or None, not list